# GraphCast con datos ocenaográficos
---
Se intenta cambiar algunas variables de entrada del modelo GraphCast por otras variables oceanográficas para ver porbar la posibilidad de un transfer learning.
  

In [4]:
import copernicusmarine

In [5]:
# Este módulo proporciona un decorador y funciones para crear clases con campos de datos, 
# similar a collections.namedtuple.
import dataclasses
# Este módulo proporciona clases para manipular fechas y horas.
import datetime
# Este módulo proporciona funciones de orden superior y operaciones de datos en general.
import functools
# Este módulo proporciona funciones matemáticas.
import math
# Este módulo proporciona operaciones de coincidencia de expresiones regulares.
import re
# Este módulo proporciona soporte para tipos opcionales.
from typing import Optional
# Este paquete proporciona funcionalidad para trabajar con sistemas de referencia de coordenadas en Cartopy, 
# una biblioteca de Python para dibujar mapas geoespaciales.
import cartopy.crs as ccrs
# Este paquete proporciona funcionalidad para interactuar con Google Cloud Storage, 
# un servicio de almacenamiento en la nube de Google.
from google.cloud import storage
# Este módulo proporciona implementaciones de modelos autoregresivos para series temporales.
from graphcast import autoregressive
# Este módulo proporciona funcionalidad para realizar operaciones de casting en datos.
from graphcast import casting
# Este módulo proporciona funcionalidad para el control de puntos de control 
# durante la ejecución de un programa.
from graphcast import checkpoint
# Este módulo proporciona utilidades para el manejo de datos en Graphcast.
from graphcast import data_utils
# Este módulo proporciona la implementación principal de Graphcast.
from graphcast import graphcast
# Este módulo proporciona funcionalidad para normalizar datos en Graphcast.
from graphcast import normalization
# Este módulo proporciona funcionalidad para realizar operaciones de rollout en Graphcast.
from graphcast import rollout
# Este módulo proporciona funcionalidad para trabajar con arreglos multidimensionales etiquetados 
# y compatibles con JAX.
from graphcast import xarray_jax
# Este módulo proporciona funcionalidad para trabajar con estructuras de árbol 
# de arreglos multidimensionales etiquetados.
from graphcast import xarray_tree
# Esta clase proporciona funcionalidad para mostrar HTML en el contexto de IPython.
from IPython.display import HTML
# Este paquete proporciona interactividad en el notebook de IPython.
import ipywidgets as widgets
# Este paquete proporciona una biblioteca para construir redes neuronales en JAX.
import haiku as hk
# Este paquete proporciona funcionalidad para realizar cálculos numéricos de alto rendimiento 
# en dispositivos acelerados por GPU y CPU.
import jax
# Este paquete proporciona funcionalidad para la visualización de datos en Python.
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
# Este paquete proporciona funcionalidad para trabajar con arreglos y matrices en Python.
import numpy as np
# Este paquete proporciona funcionalidad para trabajar con arreglos multidimensionales etiquetados.
import xarray


In [6]:
def parse_file_parts(file_name):
    """
    Parse a file name into parts separated by underscores and hyphens.

    Args:
        file_name (str): The name of the file to parse.

    Returns:
        dict: A dictionary containing the parsed parts of the file name. The keys are the parts before the first hyphen,
              and the values are the parts after the first hyphen in each section of the file name.

    Example:
        >>> parse_file_parts("example_part1-part2_part3-part4.txt")
        {'example_part1': 'part2', 'part3': 'part4'}
    """
    return dict(part.split("-", 1) for part in file_name.split("_"))

In [7]:
# Crear un cliente de Google Cloud Storage anónimo.
gcs_client = storage.Client.create_anonymous_client()

# Obtener el bucket "dm_graphcast" del cliente de Google Cloud Storage.
gcs_bucket = gcs_client.get_bucket("dm_graphcast")

## Cargamos Modelo preentrenado
---

In [8]:
# Cargar el modelo

# Obtener la fuente seleccionada en las pestañas de selección.
params_file = 'GraphCast_small - ERA5 1979-2015 - resolution 1.0 - pressure levels 13 - mesh 2to5 - precipitation input and output.npz'
# Cargar los parámetros del archivo seleccionado.
with gcs_bucket.blob(f"params/{params_file}").open("rb") as f:
    ckpt = checkpoint.load(f, graphcast.CheckPoint)
params = ckpt.params
state = {}

model_config = ckpt.model_config
task_config = ckpt.task_config
# Imprimir descripción y licencia del modelo cargado.
print("Descripción del modelo:\n", ckpt.description, "\n")
print("Licencia del modelo:\n", ckpt.license, "\n")

# Devolver la configuración del modelo.
model_config

Descripción del modelo:
 
Low resolution version of the GraphCast model (1deg, smaller mesh), with 37
pressure levels. This model is trained on ERA5 data from 1979 to 2015, and can
be causally evaluated on 2016 and later years. This model takes as inputs
`total_precipitation_6hr`. This model has much lower memory requirements.
 

Licencia del modelo:
 
The model weights are licensed under the Creative Commons
Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0). You
may obtain a copy of the License at:
https://creativecommons.org/licenses/by-nc-sa/4.0/.
The weights were trained on ERA5 data, see README for attribution statement.
 



ModelConfig(resolution=1.0, mesh_size=5, latent_size=512, gnn_msg_steps=16, hidden_layers=1, radius_query_fraction_edge_length=0.6, mesh2grid_edge_normalization_factor=0.6180338738074472)

In [9]:
def data_valid_for_model(
        file_name: str, model_config: graphcast.ModelConfig, task_config: graphcast.TaskConfig):
    """
    Verificar si los datos del archivo son válidos para el modelo y la tarea configurados.

    Args:
        file_name (str): El nombre del archivo de datos.
        model_config (graphcast.ModelConfig): La configuración del modelo.
        task_config (graphcast.TaskConfig): La configuración de la tarea.

    Returns:
        bool: True si los datos son válidos para el modelo y la tarea configurados, False de lo contrario.

    """
    file_parts = parse_file_parts(file_name.removesuffix(".nc"))
    return (
        model_config.resolution in (0, float(file_parts["res"])) and
        len(task_config.pressure_levels) == int(file_parts["levels"]) and
        (
            ("total_precipitation_6hr" in task_config.input_variables and
             file_parts["source"] in ("era5", "fake")) or
            ("total_precipitation_6hr" not in task_config.input_variables and
             file_parts["source"] in ("hres", "fake"))
        )
    )

## Cargamos un batch de datos
---

In [10]:
# Cargar datos meteorológicos
dataset_file = 'source-era5_date-2022-01-01_res-1.0_levels-13_steps-12.nc'
# Verificar si el archivo de conjunto de datos es válido para el modelo y la tarea configurados.
if not data_valid_for_model(dataset_file, model_config, task_config):
    raise ValueError(
        "Archivo de conjunto de datos no válido, vuelva a ejecutar la celda anterior y elija un archivo de conjunto de datos válido.")

# Cargar el conjunto de datos meteorológicos desde el archivo seleccionado.
with gcs_bucket.blob(f"dataset/{dataset_file}").open("rb") as f:
    example_batch = xarray.load_dataset(f).compute()

# Asegurarse de que el conjunto de datos tenga al menos 3 dimensiones de tiempo (2 para entrada, al menos 1 para objetivos).
assert example_batch.dims["time"] >= 3  

# Imprimir información sobre el archivo de conjunto de datos seleccionado.
print(", ".join([f"{k}: {v}" for k, v in parse_file_parts(
    dataset_file.removesuffix(".nc")).items()]))

# Mostrar el conjunto de datos cargado.
example_batch

source: era5, date: 2022-01-01, res: 1.0, levels: 13, steps: 12


<xarray.Dataset>
Dimensions:                       (lon: 360, lat: 181, time: 14, level: 13,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 0.0 1.0 2.0 ... 358.0 359.0
  * lat                           (lat) float32 -90.0 -89.0 -88.0 ... 89.0 90.0
  * time                          (time) timedelta64[ns] 0 days 00:00:00 ... ...
  * level                         (level) int32 50 100 150 200 ... 850 925 1000
    datetime                      (batch, time) datetime64[ns] 2022-01-01 ......
Dimensions without coordinates: batch
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 2.735e+04 ... -0.07617
    land_sea_mask                 (lat, lon) float32 1.0 1.0 1.0 ... 0.0 0.0 0.0
    2m_temperature                (batch, time, lat, lon) float32 250.7 ... 2...
    mean_sea_level_pressure       (batch, time, lat, lon) float32 9.931e+04 ....
    10m_v_component_of_wind       (batch, time, lat, lon) float32 -0.4393 ......
    10m_u_component_of_wind       (batch, time, lat, lon) float32 1.309 ... -...
    ...                            ...
    temperature                   (batch, time, level, lat, lon) float32 238....
    geopotential                  (batch, time, level, lat, lon) float32 1.98...
    u_component_of_wind           (batch, time, level, lat, lon) float32 -0.0...
    v_component_of_wind           (batch, time, level, lat, lon) float32 0.00...
    vertical_velocity             (batch, time, level, lat, lon) float32 -0.0...
    specific_humidity             (batch, time, level, lat, lon) float32 2.89...

In [11]:
# Solo se corre una ves para crear el archivo de credenciales 
# copernicusmarine.login(username=username, password=password)

In [21]:
# Load xarray dataset
GLOBAL_MULTIYEAR_PHY = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_phy_my_0.083deg_P1M-m",
                                                     dataset_version="202311",
                                                     variables=["mlotst",  "thetao", "uo", "vo",],
                                                     minimum_longitude=-180,
                                                     maximum_longitude=180,
                                                     minimum_latitude=-80,
                                                     maximum_latitude=90,
                                                     start_datetime="2020-05-01T00:00:00",
                                                     end_datetime="2021-06-01T00:00:00",
                                                     minimum_depth=0.49402499198913574,
                                                     maximum_depth=1000,
                                                     )


# Print loaded dataset information
GLOBAL_MULTIYEAR_PHY

INFO - 2024-02-21T09:33:00Z - You forced selection of dataset version "202311"
INFO - 2024-02-21T09:33:00Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-02-21T09:33:01Z - Service was not specified, the default one was selected: "arco-geo-series"


<xarray.Dataset>
Dimensions:    (depth: 35, latitude: 2041, longitude: 4320, time: 14)
Coordinates:
  * depth      (depth) float32 0.494 1.541 2.646 3.819 ... 643.6 763.3 902.3
  * latitude   (latitude) float32 -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
  * longitude  (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
  * time       (time) datetime64[ns] 2020-05-01 2020-06-01 ... 2021-06-01
Data variables:
    mlotst     (time, latitude, longitude) float32 ...
    thetao     (time, depth, latitude, longitude) float32 ...
    uo         (time, depth, latitude, longitude) float32 ...
    vo         (time, depth, latitude, longitude) float32 ...
Attributes: (12/14)
    Conventions:                   CF-1.6
    area:                          GLOBAL
    contact:                       servicedesk.cmems@mercator-ocean.eu
    credit:                        E.U. Copernicus Marine Service Information...
    dataset:                       global-reanalysis-001-030-monthly
    institution:                   Mercator Ocean
    ...                            ...
    product:                       GLOBAL_REANALYSIS_001_030
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...
    references:                    http://marine.copernicus.eu
    source:                        MERCATOR GLORYS12V1
    title:                         Monthly mean fields for product GLOBAL_REA...

In [36]:
temperature = GLOBAL_MULTIYEAR_PHY.isel(depth=slice(0, 13)).isel(latitude=slice(0, 181), longitude=slice(0, 360))["thetao"]#.expand_dims("batch")

In [43]:
# Agrega batch
temp_arr = np.expand_dims(temperature.to_numpy(), axis=0)

In [44]:
temp_arr.shape

(1, 14, 13, 181, 360)

Suplantamos los datos de temperatura atmosféricos por los marinos.

In [49]:
example_batch["temperature"].data = temp_arr

In [50]:
example_batch

<xarray.Dataset>
Dimensions:                       (lon: 360, lat: 181, time: 14, level: 13,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 0.0 1.0 2.0 ... 358.0 359.0
  * lat                           (lat) float32 -90.0 -89.0 -88.0 ... 89.0 90.0
  * time                          (time) timedelta64[ns] 0 days 00:00:00 ... ...
  * level                         (level) int32 50 100 150 200 ... 850 925 1000
    datetime                      (batch, time) datetime64[ns] 2022-01-01 ......
Dimensions without coordinates: batch
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 2.735e+04 ... -0.07617
    land_sea_mask                 (lat, lon) float32 1.0 1.0 1.0 ... 0.0 0.0 0.0
    2m_temperature                (batch, time, lat, lon) float32 250.7 ... 2...
    mean_sea_level_pressure       (batch, time, lat, lon) float32 9.931e+04 ....
    10m_v_component_of_wind       (batch, time, lat, lon) float32 -0.4393 ......
    10m_u_component_of_wind       (batch, time, lat, lon) float32 1.309 ... -...
    ...                            ...
    temperature                   (batch, time, level, lat, lon) float32 nan ...
    geopotential                  (batch, time, level, lat, lon) float32 1.98...
    u_component_of_wind           (batch, time, level, lat, lon) float32 -0.0...
    v_component_of_wind           (batch, time, level, lat, lon) float32 0.00...
    vertical_velocity             (batch, time, level, lat, lon) float32 -0.0...
    specific_humidity             (batch, time, level, lat, lon) float32 2.89...